## Scraper toggle for Colab or Local

## You have to change "LOCAL" below to "COLAB" if you want to run this on colab

### Note: Our team ran this locally, and it takes a very long time to scrape all the reviews

In [ ]:
# =========================
# Cell 0: Environment Mode
# =========================
# Purpose:
# - Choose where you’re running: your local machine (GUI Chrome) or Google Colab (headless).
# - In COLAB, install Chrome/Chromedriver only if missing.
# - Optionally mount Google Drive and set the output base there.

MODE = "LOCAL"         # change to "COLAB" when running in Google Colab
OUTPUT_BASE = "./beeradvocate_out"   # will be used later; override after mounting Drive if desired

if MODE == "COLAB":
    import os, shutil, subprocess, sys

    def sh(cmd):
        print(f"$ {cmd}")
        out = subprocess.check_output(cmd, shell=True, text=True, stderr=subprocess.STDOUT)
        print(out)

    # Check if Chrome and Chromedriver are already present
    chrome_ok = shutil.which("google-chrome") is not None
    cdriver_ok = os.path.exists("/usr/local/bin/chromedriver")

    if not chrome_ok or not cdriver_ok:
        print("Chrome or Chromedriver missing; installing...")
        sh("apt-get update")
        sh("apt-get install -y wget unzip")
        sh("wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb")
        sh("apt install -y ./google-chrome-stable_current_amd64.deb")
        # Match Chrome version
        chrome_version = subprocess.check_output("google-chrome --version | awk '{print $3}'", shell=True, text=True).strip()
        url = f"https://storage.googleapis.com/chrome-for-testing-public/{chrome_version}/linux64/chromedriver-linux64.zip"
        sh(f"wget -q '{url}' -O /tmp/chromedriver.zip")
        sh("unzip -o /tmp/chromedriver.zip -d /tmp")
        sh("mv -f /tmp/chromedriver-linux64/chromedriver /usr/local/bin/chromedriver")
        sh("chmod +x /usr/local/bin/chromedriver")
    else:
        print("Chrome + Chromedriver already installed; skipping setup.")

    # Optional: mount Drive for persistent outputs
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        # Example: uncomment to write outputs to Drive
        # OUTPUT_BASE = "/content/drive/MyDrive/beeradvocate_out"
    except Exception:
        pass

print(f"MODE: {MODE}")
print(f"OUTPUT_BASE: {OUTPUT_BASE}")


## Basic Setup
### Note: my login information to beeradvocate.com is hard-coded here, please don't abuse it

In [ ]:
# =========================
# Cell 1: Setup & Helpers
# =========================
# Purpose:
# - Import libraries and set constants (URLs, creds).
# - Start Chrome differently for LOCAL vs COLAB.
# - Configure shorter timeouts to avoid long hangs.
# - Enable CDP navigation (go()) to avoid ChromeDriver read timeouts.
# - Provide small helpers for interacting with the login form reliably.

import os, re, time, random
import pandas as pd
from urllib.parse import urljoin, urlsplit, urlunsplit, urlencode

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# URLs and credentials
LOGIN_URL     = "https://www.beeradvocate.com/community/login/"
TOP_RATED_URL = "https://www.beeradvocate.com/beer/top-rated/"
BA_USER = os.environ.get("BA_USER", "ahmerrill@gmail.com")
BA_PASS = os.environ.get("BA_PASS", "beerproject123$")

def build_chrome_options(mode: str) -> webdriver.ChromeOptions:
    """Create ChromeOptions tuned for reliability. COLAB uses headless; LOCAL is visible."""
    opts = webdriver.ChromeOptions()
    if mode == "COLAB":
        opts.add_argument("--headless=new")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--window-size=1280,900")
    # Return after DOMContentLoaded; faster and avoids waiting on slow subresources
    opts.set_capability("pageLoadStrategy", "eager")
    return opts

def start_driver(mode: str) -> webdriver.Chrome:
    """Start Chrome for LOCAL or COLAB and set sane timeouts + enable CDP Page domain."""
    options = build_chrome_options(mode)
    if mode == "LOCAL":
        # Try common paths for a locally installed chromedriver
        chromedriver_paths = [
            "/usr/local/bin/chromedriver",
            "/opt/homebrew/bin/chromedriver",
            "./chromedriver",
            "chromedriver",
        ]
        last_err = None
        drv = None
        for path in chromedriver_paths:
            try:
                service = Service(path)
                drv = webdriver.Chrome(service=service, options=options)
                break
            except Exception as e:
                last_err = e
        if drv is None:
            raise RuntimeError(f"Could not start Chrome locally: {last_err}")
    else:
        # In Colab, we install to /usr/local/bin/chromedriver in Cell 0
        service = Service("/usr/local/bin/chromedriver")
        drv = webdriver.Chrome(service=service, options=options)

    # Avoid long hangs on bad pages or scripts
    drv.set_page_load_timeout(45)   # we mostly avoid driver.get, but set for safety
    drv.set_script_timeout(30)      # JS execution timeouts

    # Shorten the ChromeDriver HTTP client timeouts (non-deprecated fields)
    cfg = drv.command_executor._client_config
    cfg.timeout = 45
    cfg.keep_alive = False

    # Enable CDP Page domain so we can navigate without driver.get
    try:
        drv.execute_cdp_cmd("Page.enable", {})
    except Exception:
        pass
    return drv

# Create the driver using MODE from Cell 0
driver = start_driver(MODE)
print("Chrome started successfully.")

# -------- Helpers for the login form and reliable input filling

def click_registered_radio(driver):
    """Toggle the 'I have a password' radio with multiple selectors (XenForo variants)."""
    candidates = [
        (By.ID, "ctrl_registered"),
        (By.CSS_SELECTOR, "input[type='radio'][name='register'][id*='registered']"),
        (By.XPATH, "//input[@type='radio' and @name='register' and contains(@id,'registered')]"),
        (By.XPATH, "//*[@id='pageLogin']//label[contains(., 'Yes, my password')]"),
    ]
    for by, sel in candidates:
        try:
            el = WebDriverWait(driver, 6).until(EC.element_to_be_clickable((by, sel)))
            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
            el.click()
            time.sleep(0.4)
            return True
        except Exception:
            pass
    return False

def first_displayed(drv, locators):
    """Return the first element among locators that exists and is visible."""
    for by, sel in locators:
        try:
            els = drv.find_elements(by, sel)
            for el in els:
                if el.is_displayed():
                    return el
        except Exception:
            pass
    return None

def set_value_with_verification(drv, el, text):
    """Type into an input; fall back to JS if blocked; verify the value stuck."""
    try:
        drv.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
        WebDriverWait(drv, 5).until(EC.element_to_be_clickable(el))
    except Exception:
        pass
    try:
        el.clear()
        el.click()
        el.send_keys(text)
        time.sleep(0.3)
    except Exception:
        # Force visibility/state and set via JS + dispatch typical form events
        drv.execute_script("""
            arguments[0].style.display='block';
            arguments[0].style.visibility='visible';
            arguments[0].disabled=false;
            arguments[0].readOnly=false;
        """, el)
        drv.execute_script("arguments[0].value = arguments[1];", el, text)
        for evt in ("input", "change", "blur"):
            drv.execute_script(
                "arguments[0].dispatchEvent(new Event(arguments[1], {bubbles:true}))", el, evt
            )
        time.sleep(0.2)
    return (el.get_attribute("value") or "") == text

def go(drv, url, attempts=2, pause=0.6, dom_timeout=20, wait_xpath=None):
    """
    Navigate via Chrome DevTools Protocol (Page.navigate) instead of driver.get.
    - attempts: retry count if navigation/ready check fails
    - pause: delay between attempts
    - dom_timeout: wait for document.readyState in ('interactive','complete')
    - wait_xpath: optional XPath to enforce presence of an element before returning
    Returns True on success, False if all attempts fail.
    """
    for i in range(1, attempts + 1):
        try:
            drv.execute_cdp_cmd("Page.navigate", {"url": url})
            WebDriverWait(drv, dom_timeout).until(
                lambda d: d.execute_script("return document.readyState") in ("interactive", "complete")
            )
            if wait_xpath:
                WebDriverWait(drv, dom_timeout).until(
                    EC.presence_of_element_located((By.XPATH, wait_xpath))
                )
            return True
        except Exception as e:
            print(f"[go] attempt {i}/{attempts} failed: {e}")
            try:
                drv.execute_cdp_cmd("Page.stopLoading", {})
            except Exception:
                pass
            try:
                drv.execute_script("window.stop();")
            except Exception:
                pass
            time.sleep(pause + random.uniform(0, 0.3))
    return False


## Login

In [ ]:
# =======================
# Cell 2: Robust Login
# =======================
# Purpose:
# - Open the login page via CDP go()
# - Dismiss cookie banners
# - Toggle the "I have a password" radio
# - Fill username/password with verification
# - Submit and wait until you’ve left the login URL

def robust_login(driver, user, pwd):
    if not go(driver, LOGIN_URL, attempts=2, dom_timeout=20, wait_xpath="//*[@id='pageLogin']"):
        raise RuntimeError("Could not open login page")

    time.sleep(1.0)  # allow any overlays to render

    # Best-effort: dismiss cookie banners
    for sel in [
        "button[id*='cookie']",
        "button[class*='cookie']",
        "button[class*='accept']",
        "button[class*='agree']",
        "button[class*='consent']",
        "#onetrust-accept-btn-handler",
        ".cookie-consent button",
    ]:
        try:
            WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, sel))).click()
            time.sleep(0.2)
            break
        except Exception:
            pass

    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "pageLogin")))

    toggled = click_registered_radio(driver)
    print(f"Registered radio toggled: {toggled}")

    username_locs = [
        (By.CSS_SELECTOR, "#LoginControl"),
        (By.CSS_SELECTOR, "#ctrl_pageLogin_login"),
        (By.CSS_SELECTOR, "input[name='login']"),
    ]
    password_locs = [
        (By.CSS_SELECTOR, "#ctrl_password"),
        (By.CSS_SELECTOR, "#ctrl_pageLogin_password"),
        (By.CSS_SELECTOR, "input[type='password'][name='password']"),
    ]

    user_el = first_displayed(driver, username_locs) or next(
        (driver.find_element(by, sel) for by, sel in username_locs if driver.find_elements(by, sel)), None
    )
    pass_el = first_displayed(driver, password_locs) or next(
        (driver.find_element(by, sel) for by, sel in password_locs if driver.find_elements(by, sel)), None
    )

    if not user_el or not pass_el:
        raise RuntimeError("Could not locate username/password fields")

    ok_user = set_value_with_verification(driver, user_el, user)
    ok_pass = set_value_with_verification(driver, pass_el, pwd)
    if not (ok_user and ok_pass):
        raise RuntimeError("Inputs never retained values")

    try:
        form = user_el.find_element(By.XPATH, "ancestor::form")
    except Exception:
        form = driver.find_element(By.XPATH, "//*[@id='pageLogin']//form")

    driver.execute_script("arguments[0].dispatchEvent(new Event('submit', {bubbles:true}));", form)
    try:
        form.submit()
    except Exception:
        pass

    WebDriverWait(driver, 20).until_not(EC.url_matches(r"/community/login($|/)"))
    return driver.current_url

# Execute the login once
dest = robust_login(driver, BA_USER, BA_PASS)
print("Login successful ->", dest)


## Navigate to top-rated page and find all beers

In [ ]:
# ==================================
# Cell 3: Open "Top Rated" listings
# ==================================
# Purpose:
# - Navigate to the Top Rated page
# - Verify it loaded (not a CF interstitial)
# - Extract up to 250 unique beer links of the form /beer/profile/{brewery_id}/{beer_id}/

import re
from urllib.parse import urljoin, urlsplit

def top_rated_ready(d):
    """Heuristics to confirm we’re on the real Top Rated page."""
    try:
        html = (d.page_source or "").lower()
        url  = (d.current_url or "").lower()
        if "just a moment" in html or "verifying your browser" in html:
            return False
        if "/beer/top-rated" in url:
            if d.find_elements(By.XPATH, "//h1[contains(translate(.,'TOPRATED','toprated'),'top rated')]"):
                return True
            if d.find_elements(By.CSS_SELECTOR, "ol li, table tbody tr, .ba-table tbody tr"):
                return True
        return False
    except Exception:
        return False

if not go(driver, TOP_RATED_URL, attempts=2, dom_timeout=20,
          wait_xpath="//a[starts-with(@href, '/beer/profile/')]"):
    raise RuntimeError("Failed to open the Top Rated page")

try:
    WebDriverWait(driver, 30).until(lambda d: top_rated_ready(d))
except TimeoutException:
    print("Timed out waiting for Top Rated markers; proceeding anyway.")
    print("  URL:", driver.current_url)
    print("  Title:", driver.title)

BASE = "https://www.beeradvocate.com"
pattern = re.compile(r"^/beer/profile/(\d+)/(\d+)/?$")

anchors = driver.find_elements(By.CSS_SELECTOR, "a[href^='/beer/profile/']")
seen = set()
beers = []

for a in anchors:
    href = (a.get_attribute("href") or "").strip()
    if href.startswith("/"):
        href = urljoin(BASE, href)
    parsed = urlsplit(href)
    m = pattern.match(parsed.path)
    if not m:
        continue
    brewery_id, beer_id = m.groups()
    key = (brewery_id, beer_id)
    if key in seen:
        continue
    seen.add(key)
    name = (a.text or "").strip() or (a.get_attribute("title") or "").strip()
    beers.append({
        "brewery_id": int(brewery_id),
        "beer_id": int(beer_id),
        "name": name,
        "url": f"{BASE}/beer/profile/{brewery_id}/{beer_id}/",
    })

if len(beers) > 250:
    beers = beers[:250]

print(f"Found {len(beers)} unique beers on this page.")
for i, b in enumerate(beers[:5], 1):
    print(f"  {i}. {b['name']} -> {b['url']}")


## Scraper

In [ ]:
# =======================================
# Cell 4: Scrape reviews with resume
# =======================================
# Purpose:
# - Iterate beers (up to MAX_BEERS), collect up to MAX_PER_BEER text reviews each.
# - Paginate by 20 using start=0,20,40,...
# - Save progress PER BEER as NDJSON (JSONL) so you can resume later without redoing completed beers.
# - At the end, load all NDJSON files into ONE DataFrame (df_all).
# - Optionally save a single CSV at the end.

import os
import json
import csv
import re
import time
import random
import pandas as pd
from urllib.parse import urlsplit, urlunsplit, urlencode
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

# Output locations (based on OUTPUT_BASE from Cell 0)
OUTPUT_DIR         = OUTPUT_BASE
TEMP_DIR           = os.path.join(OUTPUT_DIR, "tmp_beers")
COMPLETED_JSON     = os.path.join(OUTPUT_DIR, "completed_beers.json")
RESUME_JSON        = os.path.join(OUTPUT_DIR, "resume_offsets.json")
FINAL_CSV_PATH     = os.path.join(OUTPUT_DIR, "reviews_all.csv")

# Controls
MAX_BEERS          = 250
MAX_PER_BEER       = 100
PER_PAGE           = 20
PAGE_PAUSE         = 0.8
MAX_PAGES_PER_BEER = 20
STRICT_TEXT_ONLY   = True
SAVE_FINAL_CSV     = True   # switch to False if you don't want a CSV at the end

# Ensure directories exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(TEMP_DIR, exist_ok=True)

# ---- Small utilities: I/O
def _load_json(path, default):
    try:
        with open(path, "r", encoding="utf-8") as f:
            return json.load(f)
    except Exception:
        return default

def _atomic_write_json(path, obj):
    tmp = path + ".tmp"
    with open(tmp, "w", encoding="utf-8") as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)
    os.replace(tmp, path)

def _beer_jsonl_path(beer_id):
    return os.path.join(TEMP_DIR, f"{int(beer_id)}.jsonl")

def _append_jsonl(path, rows):
    with open(path, "a", encoding="utf-8") as f:
        for r in rows:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

def _read_all_jsonl(temp_dir):
    rows = []
    for fname in os.listdir(temp_dir):
        if not fname.endswith(".jsonl"):
            continue
        fpath = os.path.join(temp_dir, fname)
        with open(fpath, "r", encoding="utf-8") as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                try:
                    rows.append(json.loads(line))
                except Exception:
                    pass
    return rows

# ---- URL & parsing helpers
def normalize_reviews_url(base_url, start=0):
    sp = urlsplit(base_url)
    path = sp.path if sp.path.endswith('/') else sp.path + '/'
    q = urlencode({"view": "beer", "show": "recent", "start": start})
    return urlunsplit((sp.scheme, sp.netloc, path, q, "lists"))

def get_page_average(drv):
    try:
        el = drv.find_element(By.CSS_SELECTOR, "span.ba-ravg")
        m = re.search(r"\d+(?:\.\d+)?", el.text.strip())
        return float(m.group(0)) if m else None
    except Exception:
        return None

def get_review_blocks(drv):
    return drv.find_elements(By.XPATH, "//div[@id='rating_fullview_content_2']")

def extract_subscores_line(block):
    try:
        for s in block.find_elements(By.CSS_SELECTOR, "span.muted"):
            t = (s.text or "")
            if "look:" in t and "taste:" in t and "overall:" in t:
                return t.strip()
    except Exception:
        pass
    return None

def parse_subscores(text):
    out = {"look": None, "smell": None, "taste": None, "feel": None, "overall": None}
    if not text:
        return out
    def grab(label):
        m = re.search(rf"{label}\s*:\s*([\d.]+)", text, re.I)
        return float(m.group(1)) if m else None
    out["look"]    = grab("look")
    out["smell"]   = grab("smell")
    out["taste"]   = grab("taste")
    out["feel"]    = grab("feel")
    out["overall"] = grab("overall")
    return out

def extract_review(block):
    # Username
    username = None
    try:
        u = block.find_element(By.CSS_SELECTOR, ".username")
        username = (u.text or "").strip()
    except Exception:
        pass

    # User rating (prefer BAscore_norm)
    user_rating = None
    try:
        r = block.find_element(By.CSS_SELECTOR, "span.BAscore_norm")
        m = re.search(r"\d+(?:\.\d+)?", (r.text or "").strip())
        if m:
            user_rating = float(m.group(0))
    except Exception:
        pass
    if user_rating is None:
        m = re.search(r"Rated:\s*([0-9]+(?:\.[0-9]+)?)", block.text, re.I)
        if m:
            try:
                user_rating = float(m.group(1))
            except Exception:
                pass

    # Subscores
    subscores_raw = extract_subscores_line(block)
    subs = parse_subscores(subscores_raw or "")

    # Review text: longest div with body-like styling, else longest <p>
    review_text = ""
    try:
        body_candidates = block.find_elements(By.CSS_SELECTOR, "div[style*='line-height'], div[style*='font-size']")
        texts = [(b.text or "").strip() for b in body_candidates]
        review_text = max(texts, key=len) if texts else ""
        if len(review_text) < 15:
            ps = [p.text.strip() for p in block.find_elements(By.TAG_NAME, "p")]
            if ps:
                review_text = max(ps, key=len)
    except Exception:
        pass
    if STRICT_TEXT_ONLY and not review_text.strip():
        return None

    # Date
    date_txt = None
    try:
        muted_spans = block.find_elements(By.CSS_SELECTOR, "span.muted")
        for s in reversed(muted_spans):
            a = s.find_elements(By.TAG_NAME, "a")
            if a:
                candidate = (a[0].text or "").strip()
                if candidate:
                    date_txt = candidate
                    break
        if not date_txt:
            times = block.find_elements(By.TAG_NAME, "time")
            if times:
                date_txt = (times[0].get_attribute("datetime") or times[0].text or "").strip()
    except Exception:
        pass

    return {
        "username": username,
        "user_rating": user_rating,
        "look": subs["look"],
        "smell": subs["smell"],
        "taste": subs["taste"],
        "feel": subs["feel"],
        "overall": subs["overall"],
        "date": date_txt,
        "review_text": review_text
    }

# ---- Resume state
completed_beers = set(_load_json(COMPLETED_JSON, []))
resume_offsets  = _load_json(RESUME_JSON, {})

# ---- Final columns (DataFrame schema)
CSV_FIELDS = [
    "beer","average_user_rating","username","user_rating","delta_from_average",
    "look","smell","taste","feel","overall","date","review_text",
    "brewery_id","beer_id","page_start"
]

total_beers = min(MAX_BEERS, len(beers))
print(f"Starting scrape for {total_beers} beers. Temp files: {TEMP_DIR}")

for idx, beer in enumerate(beers[:total_beers], start=1):
    name = beer.get("name") or ""
    brewery_id = int(beer.get("brewery_id"))
    beer_id = int(beer.get("beer_id"))
    url = beer.get("url")
    if not url:
        continue

    # Already completed?
    if beer_id in completed_beers:
        print(f"[{idx}/{total_beers}] {name}: already completed — skipping")
        continue

    # Resume offset for this beer if present
    start = int(resume_offsets.get(str(beer_id), 0))
    per_beer_count = 0
    pages_seen = 0
    avg_rating = None
    last_url = None
    jsonl_path = _beer_jsonl_path(beer_id)

    print(f"[{idx}/{total_beers}] {name}:", end=" ", flush=True)

    while per_beer_count < MAX_PER_BEER and pages_seen < MAX_PAGES_PER_BEER:
        page_url = normalize_reviews_url(url, start=start)
        ok = go(driver, page_url, attempts=2, dom_timeout=20,
                wait_xpath="//div[@id='rating_fullview_content_2']")
        if not ok:
            print(" navigation failed; stopping beer")
            break

        cur_url = driver.current_url
        if last_url == cur_url:
            print(" same URL; stopping beer")
            break
        last_url = cur_url

        time.sleep(PAGE_PAUSE + random.uniform(0.0, 0.3))

        if avg_rating is None:
            avg_rating = get_page_average(driver)

        blocks = get_review_blocks(driver)
        total_blocks = len(blocks)

        page_rows = []
        for b in blocks:
            r = extract_review(b)
            if r is None:
                continue
            r["beer"] = name
            r["average_user_rating"] = avg_rating
            r["delta_from_average"] = (
                (r["user_rating"] - avg_rating) if (avg_rating is not None and r["user_rating"] is not None) else None
            )
            r["brewery_id"] = brewery_id
            r["beer_id"] = beer_id
            r["page_start"] = start
            page_rows.append(r)
            per_beer_count += 1
            if per_beer_count >= MAX_PER_BEER:
                break

        # Append this page’s rows to the beer’s NDJSON
        if page_rows:
            _append_jsonl(jsonl_path, page_rows)

        # Update resume offset for this beer
        resume_offsets[str(beer_id)] = start
        _atomic_write_json(RESUME_JSON, resume_offsets)

        print(".", end="", flush=True)
        pages_seen += 1

        if total_blocks < PER_PAGE:
            break
        start += PER_PAGE

    # Mark beer as completed if we got any rows (or if we hit terminal condition)
    if os.path.exists(jsonl_path) and os.path.getsize(jsonl_path) > 0:
        completed_beers.add(beer_id)
        _atomic_write_json(COMPLETED_JSON, sorted(list(completed_beers)))
        # Clear resume offset for this beer now that it is complete
        if str(beer_id) in resume_offsets:
            resume_offsets.pop(str(beer_id), None)
            _atomic_write_json(RESUME_JSON, resume_offsets)

    print(f" {per_beer_count} reviews")

# ---- Build final DataFrame from all per-beer NDJSON files
rows = _read_all_jsonl(TEMP_DIR)
if rows:
    df_all = pd.DataFrame(rows, columns=CSV_FIELDS)
else:
    df_all = pd.DataFrame(columns=CSV_FIELDS)

print(f"\nAssembled DataFrame with {len(df_all)} rows and {len(df_all.columns)} columns.")

# ---- Optional: save one CSV at the end
if SAVE_FINAL_CSV:
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    df_all.to_csv(FINAL_CSV_PATH, index=False)
    print(f"Final CSV written to: {FINAL_CSV_PATH}")

# Quick peek
display(df_all.head(10))
